<a href="https://colab.research.google.com/github/OMGarad/BERT-Project/blob/main/Extending_Sentence_Embedding_To_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 2.6 MB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 37.9 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 53.5 MB/s 
     |████████████████████████████████| 636 kB 46.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=d2c95b97bb8b7c9124d2a1e4fe02a84580c86a4d3b5f3426d4191882b007af07
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from torch import nn

In [ ]:
from sentence_transformers import SentenceTransformer, models

#Creating our own teacher and student sentence embedding models

max_seq_length = 128 #maximum length of the sentence for input (input greater than 128 characters will be truncated)
train_batch_size = 64

# Load teacher model
print("Load teacher model")
teacher_model = SentenceTransformer('bert-base-nli-stsb-mean-tokens') #The teacher model is bert-base-nli-stsb-mean-tokens

# Create student model
print("Create student model")
word_embedding_model = models.Transformer("xlm-roberta-base") #The student model is xlm-roberta-base

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

# Creating a dense layer in order to reduce the dimesions 
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())


model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


Load teacher model


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
import os, datetime

In [ ]:
print(os.getcwd())

/content


##Loading the train, test and validation datasets

In [ ]:
from sentence_transformers.datasets import ParallelSentencesDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentencesDataset, losses, evaluation, readers


###### Load train sets ######

train_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
train_reader.load_data('Translated_Dataset_Testing.txt') #Translated dataset containing English text and the Hindi translation
train_dataloader = DataLoader(train_reader, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MSELoss(model=model)


###### Load dev sets ######

#evaluators = []
#sts_reader = readers.STSDataReader('dataset/', s1_col_idx=0, s2_col_idx=1, score_col_idx=2)
#dev_data = SentencesDataset(examples=sts_reader.get_examples('dev.txt'), model=model)
#dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
#evaluator_sts = evaluation.EmbeddingSimilarityEvaluator(dev_dataloader, name='dev')
#evaluators.append(evaluator_sts)


###### Load test sets ######

test_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
test_reader.load_data('Translated_Dataset_Training.txt') #Translated dataset containing English text and the Hindi translation
test_dataloader = DataLoader(test_reader, shuffle=False, batch_size=train_batch_size)
#test_mse = evaluation.MSEEvaluator(test_dataloader, name='test')
#evaluators.append(test_mse)


###### Train model ######

output_path = "tmp"
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=20,
          evaluation_steps=1000,
          warmup_steps=10000,
          scheduler='warmupconstant',
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
          )

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/85 [00:00<?, ?it/s]

Iteration:   0%|          | 0/85 [00:00<?, ?it/s]

Iteration:   0%|          | 0/85 [00:00<?, ?it/s]